In [2]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os

import pandas as pd


In [3]:
class Encoder(nn.Module):
        def __init__(self, hidden_size, num_layers):
            super(Encoder, self).__init__()        
            self.lstm = nn.LSTM(2, hidden_size, num_layers, batch_first=True)

        def forward(self, data):
            out, hidden = self.lstm(data)

            return out, hidden
        
class Seq2Seq(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, future_num_frames, seq_len, decoder_seq_len, num_layers=1, input_size=256):
        super(Seq2Seq, self).__init__()
        self.future_num_frames = future_num_frames
        self.input_size = input_size
        self.seq_len = seq_len
        self.decoder_seq_len = decoder_seq_len
        
        self.encoder_lstm = Encoder(hidden_size1, num_layers)
        
        self.decoder = nn.LSTM(input_size, hidden_size1, num_layers, batch_first=True)
        
        self.fc1 = nn.Linear(hidden_size1, hidden_size2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size2, 2)
        
    def forward(self, data, device):
        _, hidden = self.encoder_lstm(data)
        
        batch_size = data.shape[0]
        out, _ = self.decoder(torch.ones(batch_size, self.decoder_seq_len, self.input_size).to(device), hidden)
        
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out.view(batch_size, self.future_num_frames, 2)

In [4]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "A:/CSCI 566 Project/"
dm = LocalDataManager(None)
# get config
# cfg = load_config_data("./configs/Frame_LSTM_Config.yaml")
cfg = {'format_version': 4,
 'model_params': {'history_num_frames': 10,
  'history_step_size': 1,
  'history_delta_time': 0.1,
  'future_num_frames': 50,
  'future_step_size': 1,
  'future_delta_time': 0.1},
 'raster_params': {'raster_size': [224, 224],
  'pixel_size': [0.5, 0.5],
  'ego_center': [0.5, 0.5],
  'map_type': 'py_semantic',
  'satellite_map_key': 'aerial_map/aerial_map.png',
  'semantic_map_key': 'semantic_map/semantic_map.pb',
  'dataset_meta_key': 'meta.json',
  'filter_agents_threshold': 0.5,
  'disable_traffic_light_faces': False},
 'train_data_loader': {'key': 'scenes/train.zarr',
  'batch_size': 64,
  'shuffle': True,
  'num_workers': 1},
 'val_data_loader': {'key': 'scenes/validate.zarr',
  'batch_size': 64,
  'shuffle': False,
  'num_workers': 1},
 'test_data_loader': {'key': 'scenes/test.zarr',
  'batch_size': 32,
  'shuffle': False,
  'num_workers': 1},
 'train_params': {'checkpoint_every_n_steps': 250,
  'max_num_steps': 100000,
  'eval_every_n_steps': 100}}

In [29]:
# ===== INIT DATASET
train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset,
                              shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"])

FUTURE_NUM_FRAMES = cfg['model_params']['future_num_frames']
SEQ_LEN = cfg['model_params']['history_num_frames'] + 1



# ===== INIT  VAL DATASET
val_cfg = cfg["val_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Train dataset/dataloader
val_zarr = ChunkedDataset(dm.require(val_cfg["key"])).open()
val_dataset = AgentDataset(cfg, val_zarr, rasterizer)
val_dataloader = DataLoader(val_dataset,
                              shuffle=val_cfg["shuffle"],
                              batch_size=val_cfg["batch_size"])
                              #num_workers=train_cfg["num_workers"])
    
# print(train_dataset, val_dataset)

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 64
model = Seq2Seq(hidden_size, 16, FUTURE_NUM_FRAMES, SEQ_LEN, FUTURE_NUM_FRAMES).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction="none")

In [36]:
# ==== TRAIN LOOP
tr_it = iter(train_dataloader)
vl_it = iter(val_dataloader)
progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]), position=0)
checkpoint_n = cfg["train_params"]["checkpoint_every_n_steps"]
losses_train = []
curr_losses = []
losses_avg = []
hidden = None

losses_val = []
val_avg = []

num_frames = 20
for i in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    
    # Train
    model.train()
    torch.set_grad_enabled(True)
    pred = model.forward(torch.flip(data['history_positions'], [1]).to(device), device)
    
    targets = data['target_positions'].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    
    # Backward pass
    loss = criterion(pred, targets)
    loss = loss * target_availabilities
    loss = loss.mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses_train.append(loss.item())
    curr_losses.append(loss.item())
    
    with torch.no_grad():
        try:
            val_data = next(vl_it)
        except StopIteration:
            vl_it = iter(val_dataloader)
            val_data = next(vl_it)

        model.eval()
        pred = model.forward(torch.flip(val_data['history_positions'], [1]).to(device), device)
    
        targets = val_data['target_positions'].to(device)
        target_availabilities = val_data["target_availabilities"].unsqueeze(-1).to(device)

        # Backward pass
        v_loss = criterion(pred, targets)
        v_loss = v_loss * target_availabilities
        v_loss = v_loss.mean()
        losses_val.append(v_loss.item())
        
    if (i % checkpoint_n) == checkpoint_n - 1:
#     if i % 2 == 1:
        with open('Seq2Seq_LSTM_val.csv','a') as fd:
            for loss in losses_val:
                fd.write(f"{i},{loss}\n")
                
        with open('Seq2Seq_LSTM_train.csv','a') as fd:
            for loss in losses_train:
                fd.write(f"{i},{loss}\n")

        train_avg_loss = np.mean(losses_train)
        val_avg_loss =  np.mean(losses_val)
        
        losses_train = []
        losses_val = []
        
        
        with open('Seq2Seq_LSTM_train_avg.csv','a') as fd:
            fd.write(f"{i},{train_avg_loss}\n")
            
        with open('Seq2Seq_LSTM_val_avg.csv','a') as fd:
            fd.write(f"{i},{val_avg_loss}\n")
        
        progress_bar.set_description(f"loss: {loss} loss(avg): {train_avg_loss} loss_val(avg): {val_avg_loss}")
        if (i+1) % 1000 == 0:
            torch.save(model.state_dict(), f'Seq2Seq_LSTM_{i}.pth')

loss: 83.40279388427734 loss(avg): 88.69890975952148 loss_val(avg): 30.391950607299805:   0%| | 4/100000 [00:18<131:03:


KeyboardInterrupt: 

In [5]:
# Frame_Partial_LSTM9999
train_cfg = cfg["test_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset,
                              shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"])

FUTURE_NUM_FRAMES = cfg['model_params']['future_num_frames']
SEQ_LEN = cfg['model_params']['history_num_frames'] + 1

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hidden_size = 64

FUTURE_NUM_FRAMES = cfg['model_params']['future_num_frames']
SEQ_LEN = cfg['model_params']['history_num_frames'] + 1

model = Seq2Seq(hidden_size, 16, FUTURE_NUM_FRAMES, SEQ_LEN, FUTURE_NUM_FRAMES).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss(reduction="none")

model.load_state_dict(torch.load('saved_models/Seq2Seq_LSTM_2999.pth'))

<All keys matched successfully>

In [10]:
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(1000), position=0)
checkpoint_n = cfg["train_params"]["checkpoint_every_n_steps"]
losses_train = []
curr_losses = []
losses_avg = []

losses_val = []
val_avg = []

hidden = None

num_frames = 20
for i in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    
    # Train
    model.eval()
    with torch.no_grad():
        pred = model.forward(torch.flip(data['history_positions'], [1]).to(device), device)
    
        targets = data['target_positions'].to(device)
        target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)

        # Backward pass
        loss = criterion(pred, targets)
        loss = loss * target_availabilities
        loss = loss.mean()
        
        losses_train.append(loss.item())
    
with open('Seq2Seq_LSTM_test.csv','a') as fd:
    for loss in losses_train:
        fd.write(f"{i},{loss}\n")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [21:17<00:00,  1.28s/it]


In [11]:
print(f"Mean Test Loss: {np.mean(losses_train)}")

Mean Test Loss: 2.290573328493774
